# Import and Load Data

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
from IPython.display import display, Math

## modules I am still trying to learn how to use
import numexpr
import bottleneck

##
pd.set_option('display.float_format', lambda x: '%.3e' % x)
np.set_printoptions(formatter={'all':lambda x: '%.3e'% x})

## Functions

In [2]:
## function to extract fits and hdf data
def load_true():
    """This is a function that takes fits and hdf data and return them to pandas dataframe"""

    # detect = fits.open('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits')[-1]
    phot   = fits.open('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits')[1]
    id_balrog = fits.open('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits')

    #load true catalogs in pandas
    # detectp_table=Table.read('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits').to_pandas()
    photdf=Table.read('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits').to_pandas()
    id_balrogdf = Table.read('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits').to_pandas()
    
    photdf.drop_duplicates(['bal_id'],inplace=True)
    
    photdf.set_index('bal_id',inplace=True,verify_integrity=True)

    return(photdf)


def load_mcal():
    #all objects riz
#     mcal0= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-v1.2.h5','r')['catalog']['unsheared'] #with neighbor
#     mcal1= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-v1.2.h5','r')['catalog']['unsheared'] #no neighbor
    ##only matched balrog object riz
    mcal2=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal3=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    #only matched balrog object griz
    mcal4=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal5=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    
    mcal_list=[mcal2,mcal3,mcal4,mcal5]
    return (mcal_list)

In [3]:
## function to convert 
def get_df_list(mcal_list):
    mcal_df_list =[]
    for catalog in mcal_list:
        local_df=pd.DataFrame()
        for key in catalog.keys():
            local_df[key]=np.array(catalog[key][:]).byteswap().newbyteorder()
        if 'bal_id' in local_df.keys():
            local_df.set_index('bal_id',inplace=True,verify_integrity=True)
        mcal_df_list.append(local_df)
    return(mcal_df_list)

In [4]:
photdf=load_true()

In [5]:
mcal_df_list=get_df_list(load_mcal())

## List of dtypes in mcal

In [6]:
mcal_df_list[2].dtypes

R11                    >f8
R12                    >f8
R21                    >f8
R22                    >f8
T                  float64
T_err              float64
coadd_object_id      int64
covmat_0_1         float64
covmat_1_1         float64
covmat_2_2         float64
dec                float64
e_1                float64
e_2                float64
flags                int32
flux_err_g         float64
flux_err_i         float64
flux_err_r         float64
flux_err_z         float64
flux_g             float64
flux_i             float64
flux_r             float64
flux_z             float64
mask_frac          float64
mcal_psf_T         float64
mcal_psf_e1        float64
mcal_psf_e2        float64
nimage_tot_g         int32
nimage_tot_i         int32
nimage_tot_r         int32
nimage_tot_z         int32
nimage_use_g         int32
nimage_use_i         int32
nimage_use_r         int32
nimage_use_z         int32
psf_T              float64
psf_e1             float64
psf_e2             float64
r

## List of dtypes in phot

In [7]:
photdf.dtypes

true_id                    int32
true_number                int32
true_flags                 int32
true_obj_flags             int32
true_cm_mof_flags          int32
true_ra                  float64
true_dec                 float64
true_cm_T                float64
true_cm_T_err            float64
true_cm_s2n_r            float64
true_cm_flags              int32
true_cm_fracdev          float64
true_cm_TdByTe           float32
true_in_VHS_footprint      int32
true_mag_auto_det        float32
true_det_number            int32
true_tilename             object
meas_id                    int64
meas_number                int32
meas_ra                  float64
meas_dec                 float64
meas_fofid                 int64
meas_flags                 int32
meas_time_last_fit       float64
meas_box_size              int16
meas_obj_flags             int32
meas_mask_frac           float64
meas_psfrec_T            float64
meas_cm_flags              int32
meas_cm_T                float64
          

## sample of mcal data

In [8]:
mcal_df_list[3].head()

,R11,R12,R21,R22,T,T_err,coadd_object_id,covmat_0_1,covmat_1_1,covmat_2_2,...,nimage_use_g,nimage_use_i,nimage_use_r,nimage_use_z,psf_T,psf_e1,psf_e2,ra,size_ratio,snr
bal_id,,,,,,,,,,,,,,,,,,,,,
100304018326232,1.417e+00,8.730e-01,4.269e-01,9.959e-01,5.542e-01,5.172e-02,337,-5.923e-05,7.927e-04,1.003e-03,...,3,4,4,4,4.593e-01,5.778e-03,8.893e-03,4.575e+01,1.207e+00,2.913e+01
100304018321548,8.239e-01,-1.786e+00,1.673e-01,3.626e-01,5.978e-02,1.837e-02,345,1.073e-03,1.168e-02,1.219e-02,...,2,3,2,4,5.222e-01,2.164e-03,1.006e-02,4.618e+01,1.145e-01,2.728e+01
10030401832834,3.872e-01,-1.627e-01,-2.127e-01,6.816e-01,1.804e-01,1.910e-02,355,2.547e-04,2.476e-03,2.242e-03,...,2,4,3,4,5.185e-01,-4.169e-03,2.315e-02,4.632e+01,3.479e-01,4.120e+01
100304018328103,1.126e+00,-1.553e-01,7.442e-02,6.615e-01,1.600e+00,6.549e-02,393,1.477e-06,2.879e-04,2.866e-04,...,2,4,3,4,5.203e-01,-4.125e-03,2.447e-02,4.630e+01,3.075e+00,3.443e+01
100304018322349,1.469e+00,-2.898e+00,-1.853e+00,-4.159e+00,-4.546e-03,1.035e-03,399,3.697e-03,1.536e-02,1.576e-02,...,4,5,5,5,4.973e-01,1.787e-02,9.877e-03,4.587e+01,-9.142e-03,3.764e+02


# Data manipulation

## Constants

In [9]:
SNR_CUT = 10
SIZE_RATIO_CUT = 0.5
T_CUT = 1000

## Filter by SNR T and size_ratio

In [10]:
def filter_by_snr():
    for i,catalog in enumerate(mcal_df_list):
        mcal_df_list[i]=catalog[(catalog['snr']>SNR_CUT) & (catalog['T']<T_CUT) & (catalog['size_ratio']>SIZE_RATIO_CUT)]

In [11]:
filter_by_snr()

In [12]:
## we are getting the same number of objects here as we got previously
for mcal in mcal_df_list:
    print(len(mcal))

1287674
1816611
1008988
1554429


## Intersects paris of catalogs by bal_id. We only want objects existing in true catalog, MOF and woMOF  

In [13]:
index_2_3=mcal_df_list[0].index.intersection(mcal_df_list[1].index)
index_4_5=mcal_df_list[2].index.intersection(mcal_df_list[3].index)

index_2_3_true=index_2_3.intersection(photdf.index).sort_values()
index_4_5_true=index_4_5.intersection(photdf.index).sort_values()

In [14]:
mcal_df_list[0]=mcal_df_list[0].reindex(index_2_3_true);
mcal_df_list[1]=mcal_df_list[1].reindex(index_2_3_true);
mcal_df_list[2]=mcal_df_list[2].reindex(index_4_5_true);
mcal_df_list[3]=mcal_df_list[3].reindex(index_4_5_true);

In [15]:
true_df_list=[None]*4
for i in range(2):
    true_df_list[i]=photdf.reindex(index_2_3_true)
    true_df_list[i+2]=photdf.reindex(index_4_5_true)

In [16]:
for i,catalog in enumerate(mcal_df_list[0:4]):
    print("The lenth of mcal{} is {}.".format(i+2,len(catalog.index)))
#     print(catalog.T.isnull().any())

The lenth of mcal2 is 1098637.
The lenth of mcal3 is 1098637.
The lenth of mcal4 is 867808.
The lenth of mcal5 is 867808.


In [17]:
for i, catalog in enumerate(true_df_list):
    print ("The length of corresponding true catalog is {}.".format(len(catalog.index)))

The length of corresponding true catalog is 1098637.
The length of corresponding true catalog is 1098637.
The length of corresponding true catalog is 867808.
The length of corresponding true catalog is 867808.


## Divings catalogs into smaller catalogs only with positive g and only with negative g

In [33]:
mcal_df_list_positive=[None]*4
mcal_df_list_negative=[None]*4

index_2_3_true_positive=index_2_3_true.intersection(photdf[photdf['true_cm_g_1']>0].index.intersection(photdf[photdf['true_cm_g_2']>0].index))
index_2_3_true_negative=index_2_3_true.intersection(photdf[photdf['true_cm_g_1']<0].index.intersection(photdf[photdf['true_cm_g_2']<0].index))

index_4_5_true_positive=index_4_5_true.intersection(photdf[photdf['true_cm_g_1']>0].index.intersection(photdf[photdf['true_cm_g_2']>0].index))
index_4_5_true_negative=index_4_5_true.intersection(photdf[photdf['true_cm_g_1']<0].index.intersection(photdf[photdf['true_cm_g_2']<0].index))

for i in range(2):
    mcal_df_list_positive[i]=mcal_df_list[i].reindex(index_2_3_true_positive)
    mcal_df_list_positive[i+2]=mcal_df_list[1+2].reindex(index_4_5_true_positive)
    mcal_df_list_negative[i]=mcal_df_list[i].reindex(index_2_3_true_negative)
    mcal_df_list_negative[i+2]=mcal_df_list[1+2].reindex(index_4_5_true_negative)

# Catagory statistics

### Numbers of Objects in each catagory

In [69]:
# positive g
for catalog in mcal_df_list_positive:
    print (len(catalog))

272818
272818
215145
215145


In [68]:
# negative g
for catalog in mcal_df_list_negative:
    print (len(catalog))

270876
270876
213617
213617


# Get R'

We read $R$ matrix from the measured catalog. And we calculate $R'$ matrix with $$R_{ij}=\frac{<e_i>}{<g_j>}$$

In [34]:
def get_R(mcal_df_list):

    R_list=[None]*4
    R_prime_list=[None]*4
    
    for i,catalog in enumerate(mcal_df_list):
        true_catalog=photdf.reindex(catalog.index)

        e1=catalog['e_1'].mean()
        e2=catalog['e_2'].mean()
        g1=true_catalog['true_cm_g_1'].mean()
        g2=true_catalog['true_cm_g_2'].mean()

        R_list[i]=[[catalog['R11'].mean(),catalog['R12'].mean()],[catalog['R21'].mean(),catalog['R22'].mean()]]
        R_prime_list[i]=[[e1/g1,e1/g2],[e2/g1,e2/g2]]
                                        
    return(R_list,R_prime_list)

In [42]:
def print_R(R_list,R_prime_list):
    for i,R in enumerate(R_list):
        print ("For mcal {}".format(i+2))
        display(Math('R='))
        print(np.matrix(R))
        display(Math('R\'='))
        print(np.matrix(R_prime_list[i]))
        print("\n")

### R' for catagories without g-filtering

In [43]:
print_R(*get_R(mcal_df_list))

For mcal 2


<IPython.core.display.Math object>

[[7.769e-01 -1.044e-05]
 [-1.955e-04 7.828e-01]]


<IPython.core.display.Math object>

[[8.732e-01 2.079e+01]
 [1.138e-01 2.710e+00]]


For mcal 3


...

[[9.397e-01 5.954e+00]
 [2.031e-01 1.287e+00]]


For mcal 5


<IPython.core.display.Math object>

[[7.916e-01 1.613e-03]
 [2.058e-03 7.959e-01]]


<IPython.core.display.Math object>

[[8.985e-01 5.693e+00]
 [1.886e-01 1.195e+00]]




### R' for positive g catagories

In [44]:
print_R(*get_R(mcal_df_list_positive))

For mcal 2


<IPython.core.display.Math object>

[[7.738e-01 -6.377e-02]
 [-6.216e-02 7.828e-01]]


<IPython.core.display.Math object>

[[7.177e-01 7.467e-01]
 [6.844e-01 7.120e-01]]


For mcal 3


...

[[7.256e-01 7.599e-01]
 [6.865e-01 7.190e-01]]


For mcal 5


<IPython.core.display.Math object>

[[7.873e-01 -5.769e-02]
 [-5.774e-02 7.958e-01]]


<IPython.core.display.Math object>

[[7.256e-01 7.599e-01]
 [6.865e-01 7.190e-01]]




### R' for negative g catagories

In [45]:
print_R(*get_R(mcal_df_list_negative))

For mcal 2


<IPython.core.display.Math object>

[[7.748e-01 -5.918e-02]
 [-5.926e-02 7.802e-01]]


<IPython.core.display.Math object>

[[7.100e-01 7.247e-01]
 [6.932e-01 7.076e-01]]


For mcal 3


...

[[7.168e-01 7.344e-01]
 [6.980e-01 7.152e-01]]


For mcal 5


<IPython.core.display.Math object>

[[7.893e-01 -5.676e-02]
 [-5.743e-02 7.949e-01]]


<IPython.core.display.Math object>

[[7.168e-01 7.344e-01]
 [6.980e-01 7.152e-01]]




# Calculate g' with R matrix given in the mcal catalogs

We calculate g' (the shear after mcal correction) with the equation $$g'_{i}=\frac{<e_i>}{R_{ii}}$$

In [46]:
def get_g_prime(mcal_df_list):

    g_prime_list=[None]*4
    
    for i,catalog in enumerate(mcal_df_list):
        true_catalog=photdf.reindex(catalog.index)

        e1=catalog['e_1'].mean()
        e2=catalog['e_2'].mean()
        R11=catalog['R11'].mean()
        R22=catalog['R22'].mean()

        g_prime_list[i]=[e1/R11,e2/R22]
    return(g_prime_list)

In [57]:
def print_g_prime(g_prime_list):
    for i,g_prime in enumerate(g_prime_list):
        print ("For mcal {}".format(i+2))
        display(Math('g\'='))
        print(np.matrix(g_prime))
        print('\n')

### g' for positive g catagories

In [58]:
print_g_prime(get_g_prime(mcal_df_list_positive))

For mcal 2


<IPython.core.display.Math object>

[[1.904e-01 1.795e-01]]


For mcal 3


<IPython.core.display.Math object>

[[1.917e-01 1.802e-01]]


For mcal 4


<IPython.core.display.Math object>

[[1.878e-01 1.758e-01]]


For mcal 5


<IPython.core.display.Math object>

[[1.878e-01 1.758e-01]]




### g' for negative g catagories

In [59]:
print_g_prime(get_g_prime(mcal_df_list_positive))

For mcal 2


<IPython.core.display.Math object>

[[1.904e-01 1.795e-01]]


For mcal 3


<IPython.core.display.Math object>

[[1.917e-01 1.802e-01]]


For mcal 4


<IPython.core.display.Math object>

[[1.878e-01 1.758e-01]]


For mcal 5


<IPython.core.display.Math object>

[[1.878e-01 1.758e-01]]




#TODO
calculate error